In [1]:
# library
import torch
import numpy as np
import pandas as pd
from torchmetrics import R2Score

r2score = R2Score()

torch.manual_seed(1)
np.random.seed(1)

In [2]:
# Model
class Net(torch.nn.Module):
  def __init__(self, h1, h2):
    super(Net, self).__init__()
    self.input   = torch.nn.Linear(4, h1)
    self.hidden1 = torch.nn.Linear(h1, h2)
    self.output  = torch.nn.Linear(h2, 1)

  def forward(self, x):
    z = torch.selu(self.input(x))
    z = torch.selu(self.hidden1(z))
    z = torch.relu(self.output(z))
    return z

In [3]:
# Model evaluation
def eval(model, testset):
    with torch.no_grad():
        pred_Y = model(testset.x_data)
    
    r2 = r2score(pred_Y, testset.y_data)
    return r2.item()

In [4]:
# Data class
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, start=None, end=None):
    df = pd.read_csv(src_file)
    Tz_k   = np.array(df['room_temp']).reshape(-1,1)[start: end]
    To_k   = np.array(df['thermostat_outside_temp']).reshape(-1,1)[start: end]
    Tsa_k1 = np.array(df['supply_discharge_temp']).reshape(-1,1)[start+1: end+1]
    msa_k1 = np.array(df['airflow_current']).reshape(-1,1)[start+1: end+1]

    tmp_x = np.concatenate((Tz_k, Tsa_k1, msa_k1, To_k), axis=1)
    tmp_y = np.array(df['room_temp']).reshape(-1,1)[start+1: end+1]

    self.x_data = torch.tensor(tmp_x, dtype=torch.float32)
    self.y_data = torch.tensor(tmp_y, dtype=torch.float32)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    inp  = self.x_data[idx]
    outp = self.y_data[idx]
    sample = {'inp':inp, 'outp':outp}
    return sample

In [5]:
# Early stopping function
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.00001*np.max(list[0: -patience])):
            return 1
    return 0

In [6]:
# train function
def train(net, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100, smooth=0):
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    R2_test = np.array([])
    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=train_ds.y_data.shape[0], shuffle=True)
    for _ in range(0, max_epochs+1):
        net.train()
        for (_, batch) in enumerate(train_ldr):
            X = batch['inp']
            Y = batch['outp']

            optimizer.zero_grad()
            output = net(X)
            loss_val = loss_func(output, Y) + smooth*loss_func(output, X[:,0].reshape(-1,1))
            loss_val.backward()
            optimizer.step()

        net.eval()
        R2_test = np.append(R2_test, eval(net, test_ds))
        
        if(early_stop(list = R2_test, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2_test

In [7]:
# main
df_result = pd.DataFrame({'n_train':[], 'smooth':[], 'size':[], 'lr':[], 'best_epoch':[], 'R2':[]})
n_end = 3000

for _n_train in [32, 64, 128]:
    for _smooth in [0, 0.001, 0.01]:
        for h in [3, 4, 16]:
            for _lr in [0.0001, 0.001, 0.01]:

                # Create network
                device = torch.device("cpu")
                net = Net(h, h).to(device)

                # Create Dataset and DataLoader objects
                src_file = 'C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_106_exp_30m_20221130.csv'
                n_train  = _n_train
                train_ds = Data(src_file, start=0,       end=n_train)
                test_ds  = Data(src_file, start=n_train, end=n_end)

                # train
                R2_test = train(net, train_ds, test_ds, lr=_lr, min_epochs=300, max_epochs=50000, patience=200, smooth=_smooth)

                # results
                df_result.loc[len(df_result)] = [_n_train, _smooth, h, _lr, np.argmax(R2_test), np.max(R2_test)]

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 4,
                       ):
    print(df_result)

    n_train  smooth  size      lr  best_epoch         R2
0      32.0   0.000   3.0  0.0001     50000.0  -166.2358
1      32.0   0.000   3.0  0.0010     30303.0    -1.3908
2      32.0   0.000   3.0  0.0100        26.0  -705.1038
3      32.0   0.000   4.0  0.0001         0.0 -3502.1887
4      32.0   0.000   4.0  0.0010      9003.0 -2190.3589
5      32.0   0.000   4.0  0.0100        33.0  -893.6959
6      32.0   0.000  16.0  0.0001         0.0 -3414.8950
7      32.0   0.000  16.0  0.0010        59.0 -1534.2089
8      32.0   0.000  16.0  0.0100         0.0 -3532.0686
9      32.0   0.001   3.0  0.0001         0.0 -3532.0686
10     32.0   0.001   3.0  0.0010       138.0 -1401.3529
11     32.0   0.001   3.0  0.0100         0.0 -3532.0686
12     32.0   0.001   4.0  0.0001         0.0 -3532.0686
13     32.0   0.001   4.0  0.0010         0.0 -1778.8580
14     32.0   0.001   4.0  0.0100        31.0 -1213.1057
15     32.0   0.001  16.0  0.0001     49860.0  -103.1757
16     32.0   0.001  16.0  0.00